---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-social-network-analysis/resources/yPcBs) course resource._

---

# Assignment 4

In [16]:
import networkx as nx
import pandas as pd
import numpy as np
import pickle

---

## Part 1 - Random Graph Identification

For the first part of this assignment you will analyze randomly generated graphs and determine which algorithm created them.

In [2]:
P1_Graphs = pickle.load(open('A4_graphs','rb'))
P1_Graphs

<br>
`P1_Graphs` is a list containing 5 networkx graphs. Each of these graphs were generated by one of three possible algorithms:
* Preferential Attachment (`'PA'`)
* Small World with low probability of rewiring (`'SW_L'`)
* Small World with high probability of rewiring (`'SW_H'`)

Anaylze each of the 5 graphs and determine which of the three algorithms generated the graph.

*The `graph_identification` function should return a list of length 5 where each element in the list is either `'PA'`, `'SW_L'`, or `'SW_H'`.*

In [8]:
# from matplotlib import pyplot as plt
# import seaborn as sns
# %matplotlib notebook

In [9]:
# P1_stats = []
# for G in P1_Graphs:
#     degrees = G.degree()
#     degree_v = sorted(set(degrees.values()))
#     #print(degree_v)
#     histogram = [list(degrees[i]/float(nx.number_of_nodes(G)) for i in degree_v)]
#     ac = nx.average_clustering(G)
#     al = nx.average_shortest_path_length(G)
#     P1_stats.append((ac,al,degree_v, histogram))

In [10]:
# for s in P1_stats:
#     print(s[:2])

In [11]:
# this_G = P1_stats[4]

In [12]:
# plt.plot(this_G[2],this_G[3][0],'o')
# plt.xlabel('Degree')
# plt.ylabel('Fraction of Nodes')
# plt.xscale('log')
# plt.yscale('log')

In [13]:
def graph_identification():
    
    # Your Code Here
    
    return ['PA','SW_L','SW_L','PA','SW_H']# Your Answer Here

In [14]:
graph_identification()

['PA', 'SW_L', 'SW_L', 'PA', 'SW_H']

---

## Part 2 - Company Emails

For the second part of this assignment you will be workking with a company's email network where each node corresponds to a person at the company, and each edge indicates that at least one email has been sent between two people.

The network also contains the node attributes `Department` and `ManagementSalary`.

`Department` indicates the department in the company which the person belongs to, and `ManagementSalary` indicates whether that person is receiving a management position salary.

In [17]:
G = nx.read_gpickle('email_prediction.txt')

print(nx.info(G))

Name: 
Type: Graph
Number of nodes: 1005
Number of edges: 16706
Average degree:  33.2458


### Part 2A - Salary Prediction

Using network `G`, identify the people in the network with missing values for the node attribute `ManagementSalary` and predict whether or not these individuals are receiving a management position salary.

To accomplish this, you will need to create a matrix of node features using networkx, train a sklearn classifier on nodes that have `ManagementSalary` data, and predict a probability of the node receiving a management salary for nodes where `ManagementSalary` is missing.



Your predictions will need to be given as the probability that the corresponding employee is receiving a management position salary.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 252 with the data being the probability of receiving management salary, and the index being the node id.

    Example:
    
        1       1.0
        2       0.0
        5       0.8
        8       1.0
            ...
        996     0.7
        1000    0.5
        1001    0.0
        Length: 252, dtype: float64

In [18]:
df = pd.DataFrame(index=G.nodes())

In [19]:
df['Department'] = pd.Series(nx.get_node_attributes(G, 'Department'))
df['ManagementSalary'] = pd.Series(nx.get_node_attributes(G, 'ManagementSalary'))
df['clustering'] = pd.Series(nx.clustering(G))
df['degree'] = pd.Series(G.degree())
df['degree_cent'] = pd.Series(nx.degree_centrality(G))
df['between_cent'] = pd.Series(nx.betweenness_centrality(G))

In [20]:
h, a = nx.hits(G)
df['hub_score'] = pd.Series(h)
df['auth_score'] = pd.Series(a)

In [21]:
df.head()

,Department,ManagementSalary,clustering,degree,degree_cent,between_cent,hub_score,auth_score
0,1,0.0,0.276423,44,0.043825,0.001124,0.000944,0.000944
1,1,NaN,0.265306,52,0.051793,0.001195,0.001472,0.001472
2,21,NaN,0.297803,95,0.094622,0.006570,0.002680,0.002680
3,21,1.0,0.384910,71,0.070717,0.001654,0.002369,0.002369
4,21,1.0,0.318691,96,0.095618,0.005547,0.003055,0.003055


In [22]:
df_train = df[~np.isnan(df['ManagementSalary'])]
df_test = df[np.isnan(df['ManagementSalary'])]
X_train = df_train.drop(labels='ManagementSalary', axis=1)
X_test = df_test.drop(labels='ManagementSalary', axis=1)
y_train = df_train['ManagementSalary']
y_test = df_test['ManagementSalary']

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [24]:
scaler = StandardScaler()
scaler.fit(X_train)
X_train_new = scaler.transform(X_train)
X_test_new = scaler.transform(X_test)

In [25]:
param_dict = {'penalty':('l1','l2'),
              'C':np.power(10.0,(np.arange(-2,2,1)))}

In [26]:
lr = LogisticRegression(max_iter=500)
clf = GridSearchCV(lr, param_dict, scoring='roc_auc')
clf.fit(X_train_new, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=500, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ('l1', 'l2'), 'C': array([  0.01,   0.1 ,   1.  ,  10.  ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [27]:
y_pred = clf.predict_proba(X_test_new)[:,1]

In [28]:
clf.best_score_

0.9265755142073262

In [29]:
def salary_predictions():
    
    # Your Code Here
    
    return pd.Series(y_pred, index=y_test.index)  # Your Answer Here

In [30]:
salary_predictions()

1       0.130282
2       0.634750
5       0.998583
8       0.118887
14      0.276311
18      0.235738
27      0.272322
30      0.317480
31      0.177298
34      0.136691
37      0.104885
40      0.168022
45      0.198212
54      0.221078
55      0.264409
60      0.137563
62      0.999004
65      0.707624
77      0.101150
79      0.131782
97      0.107638
101     0.081200
103     0.148256
108     0.126150
113     0.449387
122     0.060383
141     0.414096
142     0.908694
144     0.132792
145     0.546766
          ...   
913     0.074880
914     0.106183
915     0.020885
918     0.121350
923     0.079795
926     0.115651
931     0.085797
934     0.057806
939     0.039518
944     0.024116
945     0.079796
947     0.157237
950     0.148443
951     0.078784
953     0.056232
959     0.019422
962     0.020615
963     0.162692
968     0.153859
969     0.152058
974     0.110460
984     0.056930
987     0.144569
989     0.144160
991     0.186786
992     0.020215
994     0.023717
996     0.0216

### Part 2B - New Connections Prediction

For the last part of this assignment, you will predict future connections between employees of the network. The future connections information has been loaded into the variable `future_connections`. The index is a tuple indicating a pair of nodes that currently do not have a connection, and the `Future Connection` column indicates if an edge between those two nodes will exist in the future, where a value of 1.0 indicates a future connection.

In [31]:
future_connections = pd.read_csv('Future_Connections.csv', index_col=0, converters={0: eval})
future_connections.head(10)

,Future Connection
"(6, 840)",0.0
"(4, 197)",0.0
"(620, 979)",0.0
"(519, 872)",0.0
"(382, 423)",0.0
"(97, 226)",1.0
"(349, 905)",0.0
"(429, 860)",0.0
"(309, 989)",0.0
"(468, 880)",0.0


Using network `G` and `future_connections`, identify the edges in `future_connections` with missing values and predict whether or not these edges will have a future connection.

To accomplish this, you will need to create a matrix of features for the edges found in `future_connections` using networkx, train a sklearn classifier on those edges in `future_connections` that have `Future Connection` data, and predict a probability of the edge being a future connection for those edges in `future_connections` where `Future Connection` is missing.



Your predictions will need to be given as the probability of the corresponding edge being a future connection.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC).

Your grade will be based on the AUC score computed for your classifier. A model which with an AUC of 0.88 or higher will receive full points, and with an AUC of 0.82 or higher will pass (get 80% of the full points).

Using your trained classifier, return a series of length 122112 with the data being the probability of the edge being a future connection, and the index being the edge as represented by a tuple of nodes.

    Example:
    
        (107, 348)    0.35
        (542, 751)    0.40
        (20, 426)     0.55
        (50, 989)     0.35
                  ...
        (939, 940)    0.15
        (555, 905)    0.35
        (75, 101)     0.65
        Length: 122112, dtype: float64

In [32]:
future_connections['pref att'] = [i[2] for i in nx.preferential_attachment(G, future_connections.index)]
future_connections['jaccard'] = [i[2] for i in nx.jaccard_coefficient(G, future_connections.index)]
future_connections['RA'] = [i[2] for i in nx.resource_allocation_index(G, future_connections.index)]
future_connections['num_CN'] = future_connections.index.map(
    lambda friend: len(list(nx.common_neighbors(G, friend[0], friend[1]))))

In [33]:
def get_sound_ra(nodes):
    node1, node2 = nodes
    gg = nx.ra_index_soundarajan_hopcroft(G,[(node1,node2)],community='Department')
    for i, j, k in gg:
        ra_score = k
    return ra_score

In [34]:
future_connections['sound_RA'] = future_connections.index.map(get_sound_ra)

In [35]:
def get_sound_cn(nodes):
    node1, node2 = nodes
    gg = nx.cn_soundarajan_hopcroft(G,[(node1,node2)],community='Department')
    for i, j, k in gg:
        ra_score = k
    return ra_score

In [36]:
future_connections['sound_CN'] = future_connections.index.map(get_sound_cn)

In [37]:
future_connections.head(20)

,Future Connection,pref att,jaccard,RA,num_CN,sound_RA,sound_CN
"(6, 840)",0.0,2070,0.073770,0.136721,9,0.0,9
"(4, 197)",0.0,3552,0.015504,0.008437,2,0.0,2
"(620, 979)",0.0,28,0.000000,0.000000,0,0.0,0
"(519, 872)",0.0,299,0.060606,0.039726,2,0.0,2
"(382, 423)",0.0,205,0.000000,0.000000,0,0.0,0
"(97, 226)",1.0,1575,0.048193,0.036296,4,0.0,4
"(349, 905)",0.0,240,0.000000,0.000000,0,0.0,0
"(429, 860)",0.0,816,0.000000,0.000000,0,0.0,0
"(309, 989)",0.0,184,0.000000,0.000000,0,0.0,0
"(468, 880)",0.0,672,0.019608,0.002882,1,0.0,1


In [38]:
fc_train = future_connections[~np.isnan(future_connections['Future Connection'])]
fc_test = future_connections[np.isnan(future_connections['Future Connection'])]
X_train2 = fc_train.drop(labels='Future Connection', axis=1)
X_test2 = fc_test.drop(labels='Future Connection', axis=1)
y_train2 = fc_train['Future Connection']
y_test2 = fc_test['Future Connection']

In [39]:
scaler2 = StandardScaler()
scaler2.fit(X_train2)
X_train_new2 = scaler2.transform(X_train2)
X_test_new2 = scaler2.transform(X_test2)

In [41]:
param_dict = {'C':np.power(10.0,(np.arange(-2,2,1)))}

In [43]:
lr2 = LogisticRegression()
clf2 = GridSearchCV(lr2, param_dict, scoring='roc_auc')
clf2.fit(X_train_new2, y_train2)

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': array([  0.01,   0.1 ,   1.  ,  10.  ])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=0)

In [44]:
clf2.best_score_

0.91261153549051899

In [45]:
y_pred2 = clf2.predict_proba(X_test_new2)[:,1]

In [46]:
def new_connections_predictions():
    
    # Your Code Here
    
    return pd.Series(y_pred2, index=y_test2.index)# Your Answer Here

In [47]:
new_connections_predictions()

(107, 348)    0.027112
(542, 751)    0.010521
(20, 426)     0.586194
(50, 989)     0.010579
(942, 986)    0.010643
(324, 857)    0.010571
(13, 710)     0.153359
(19, 271)     0.135388
(319, 878)    0.010600
(659, 707)    0.010527
(49, 843)     0.010618
(208, 893)    0.010541
(377, 469)    0.008262
(405, 999)    0.017879
(129, 740)    0.016639
(292, 618)    0.018944
(239, 689)    0.010604
(359, 373)    0.009164
(53, 523)     0.164240
(276, 984)    0.010615
(202, 997)    0.010627
(604, 619)    0.054345
(270, 911)    0.010603
(261, 481)    0.070340
(200, 450)    0.999991
(213, 634)    0.010503
(644, 735)    0.044072
(346, 553)    0.010356
(521, 738)    0.010076
(422, 953)    0.017556
                ...   
(672, 848)    0.010603
(28, 127)     0.934008
(202, 661)    0.010332
(54, 195)     0.999812
(295, 864)    0.010562
(814, 936)    0.010512
(839, 874)    0.010643
(139, 843)    0.010567
(461, 544)    0.009881
(68, 487)     0.009846
(622, 932)    0.010548
(504, 936)    0.016488
(479, 528) 